In [ ]:
%pylab inline

# Basic Usage

To start the simulator we import the pySibernetic module, create an owConfigProperty and an 
owPhysicsFluidSimulation objects. 

In [1]:
import pySibernetic

config=pySibernetic.owConfigProperty('worm_crawling')
config.TimeLim = 10.
config.TimeStep = 5e-6
config.OpenCLSourceFileName='src/sphFluid_crawling.cl'
config.integrationMethod=pySibernetic.EULER

sim=pySibernetic.owPhysicsFluidSimulator(config) # see output in terminal

Creating a muscle simulation

In [ ]:
m=pySibernetic.MuscleSimulation()
m.increment=1;

Defining the simulation iteration

In [ ]:
def my_sim_iteration():
    # Running the physics simulation, No Logging to file
    sim.simulationStep(False)
    #Running muscle simulation
    m_t=m.run() 
    # updating the muscle activation signal
    # We always change the array ELEMENTS and not the array itself. 
    # See comments in source code of pySibernetic.owPhysicsFluidSimulator 
    # (run: 'pySibernetic.owPhysicsFluidSimulator??' to view the source code)
    sim.MuscleAtcivationSignal[:96]=m_t 
    # pushing the C++ muscle activation signal to the correspinding OpenCL buffer
    sim.pushMuscleActivity2openCLsolver() 
    

For example let's say we are interested only in the position of the worm body. Let's create a function to get the positions of all the worm's particles  

In [ ]:
def get_worm_position():
    # reshape the numpy array to N_particles X 4. no copy
    r=sim.Position.reshape((-1,4)) 
    # return position of particles that belong to the worm (makes copy of data)
    return r[(r[:,3]>2.) * (r[:,3]<2.25)  ,:3] 

Now let's simulate T seconds and keep the position of the worm every t_save seconds

In [ ]:
T = 0.1
t_save = 0.004
n_iter = int(t_save / config.TimeStep)

worm_position = [get_worm_position()]

while sim.Iteration * config.TimeStep < T:
    # advance simulation by t_save
    for i in range(n_iter):
        my_sim_iteration()
    # get the worm's position
    worm_position.append(get_worm_position())
    # You can add other stuff here (save to files, visualzation, etc.)
    

Finally let's visualize the results using [ipyvolume](https://github.com/maartenbreddels/ipyvolume)

In [ ]:
import ipyvolume as ipv

f=ipv.figure()
ipv.xlim(0,500)
ipv.ylim(0,500)
ipv.zlim(0,500)

a=array(worm_position) 
s=ipv.scatter(a[:,:,2],a[:,:,0],a[::,:,1],size=0.1,marker='sphere')
ipv.animation_control(s,interval=100)
ipv.show()


# Exposed properties and Methods

Exposed in pySibernetic module:

In [2]:
[attr for attr in pySibernetic.__dir__() if not attr.startswith('_')]

['MuscleSimulation',
 'owConfigProperty',
 'owPhysicsFluidSimulator',
 'ALL',
 'INTEGRATOR',
 'EULER',
 'GPU',
 'LEAPFROG',
 'DEVICE',
 'getNDArrayFromBuffer',
 'CPU',
 'py_functions']

Here are the properties exposed to Python in the owConfigProperty class

In [3]:
[attr for attr in config.__dir__() if not attr.startswith('_')]

['numOfMembranes',
 'xmin',
 'NumberOfIteration',
 'SourceFileName',
 'OpenCLSourceFileName',
 'ParticleCount',
 'xmax',
 'zmin',
 'LoadPath',
 'zmax',
 'numOfLiquidP',
 'gridCellsZ',
 'ParticleCount_RoundUp',
 'ConfigPath',
 'SnapshotFileName',
 'ConfigFilePath',
 'gridCellCount',
 'DeviceName',
 'numOfElasticP',
 'TimeStep',
 'DeviceType',
 'gridCellsX',
 'ConfigFileName',
 'TimeLim',
 'ymin',
 'WormConfig',
 'LogStep',
 'gridCellsY',
 'numOfBoundaryP',
 'integrationMethod',
 'initGridCells',
 'Delta',
 'ymax',
 'prefDeviceType',
 'MUSCLE_COUNT']

Here are the properties exposed to Python in the owPhysicsFluidSimulation class

In [4]:
[attr for attr in sim.__dir__() if not attr.startswith('_')]

['ElasticConnectionsData',
 'ParticleIndex',
 'Position_cpp',
 'MuscleAtcivationSignal_cpp',
 'Velocity',
 'Density',
 'reset',
 'MuscleAtcivationSignal',
 'ElasticConnectionsData_cpp',
 'Density_cpp',
 'membraneData_cpp_size',
 'MembraneData',
 'Config',
 'Position',
 'velocity_cpp',
 'MembraneData_cpp',
 'simulationStep',
 'velocity_cpp_size',
 'ParticleIndex_cpp',
 'elasticConnectionsData_cpp_size',
 'Iteration',
 'makeSnapshot',
 'position_cpp_size',
 'muscle_activation_signal_cpp_size',
 'pushMuscleActivity2openCLsolver',
 'particleIndex_cpp_size',
 'density_cpp_size']

In [5]:
pySibernetic.owPhysicsFluidSimulator??